# USP - Introdução à Ciência de Dados

**University of São Paulo - School of Pharmaceutical Sciences of Ribeirão Preto (FCFRP)**

![https://colab.research.google.com/github/LucasVmigotto/usp-fcfrp-introducao-a-ciencia-de-dados/blob/develop/notebooks/lab.ipynb](https://colab.research.google.com/assets/colab-badge.svg)

## Sumário

* [1. Estruturas de Dados](#1-estruturas-de-dados)
* [2. Estruturas de Decisão](#2-estruturas-de-decisão)
* [3. Estruturas de Repetição](#3-estruturas-de-repetição)
* [4. Funções](#4-funções)
* [5. NumPy](#5-numpy)
* [6. Pandas](#6-pandas)
* [7. Regressão](#7-regressão)
* [8. Classificação](#8-classificação)
* [9. Agrupamento](#9-agrupamento)
* [10. Referências](#10-referências)

## 0. Setup de Ambiente

### 0.1. Instalação de libs

In [4]:
%%writefile requirements.txt
matplotlib==3.7.1
numpy==1.25.2
pandas==2.0.3
scikit-learn==1.2.2
seaborn==0.13.1

Writing requirements.txt


In [ ]:
! pip install -r requirements.txt

### 0.2. Importação de libs

In [80]:
from pathlib import Path
from urllib.request import urlretrieve

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from numpy import ndarray
from pandas import DataFrame
from requests import Response
from requests import get as req_get
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    confusion_matrix,
    r2_score,
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

### 0.3. Definição de constantes

In [3]:
# @markdown **URL dos Datasets usados**

__REGRESSAO_DATASET_URL: str = 'https://archive.ics.uci.edu/static/public/555/apartment+for+rent+classified.zip' # @param {type: "string"}
__CLASSIFICATION_DATASET_URL: str = 'https://archive.ics.uci.edu/static/public/519/heart+failure+clinical+records.zip' # @param {type: "string"}
__CLUSTERING_DATASET_URL: str = 'https://archive.ics.uci.edu/static/public/862/turkish+music+emotion.zip' # @param {type: "string"}

# @markdown ---

# @markdown **Local de armazenamento dos Datasets**
__DATA_BASE_PATH_VALUE: str = './data/' # @param {type: "string"}
__DATA_BASE_PATH: Path = Path(__DATA_BASE_PATH_VALUE)

__REGRESSION_DATA_PATH_VALUE: str = 'regression' # @param {type: "string"}
REGRESSION_DATA_PATH: Path = __DATA_BASE_PATH / __REGRESSION_DATA_PATH_VALUE

__CLASSIFICATION_DATA_PATH_VALUE: str = 'classification' # @param {type: "string"}
CLASSIFICATION_DATA_PATH: Path = __DATA_BASE_PATH / __CLASSIFICATION_DATA_PATH_VALUE

__DECOMPOSITION_DATA_PATH_VALUE: str = 'decomposition' # @param {type: "string"}
DECOMPOSITION_DATA_PATH: Path = __DATA_BASE_PATH / __DECOMPOSITION_DATA_PATH_VALUE

# @markdown ---

# @markdown **Preferências para treinamento dos modelos**

SEED: int = 42 # @param {type: "number"}
VERBOSE: bool = 42 # @param {type: "boolean"}
TEST_RATIO: int = 0.15 # @param {type: "slider", min: 0.1, max: 0.9, step: 0.05}
K_FOLDS: int = 10 # @param {type: "slider", min: 3, max: 10, step: 1}


In [4]:
np.random.seed(SEED)

## 1. Estruturas de Dados

### 1.1. Lista de temperaturas

Crie uma lista de temperaturas de [dez países](https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_extremos_meteorol%C3%B3gicos). Escreva em cada célula de um notebook o acesso a elementos da lista para calcular a temperatura em graus Fahrenheit:

$$
C=5 \cdot \frac{F - 32}{9}
$$


In [5]:
WIKIPEDIA_COUNTRIES: str = 'https://pt.wikipedia.org/wiki/Lista_de_pa%C3%ADses_por_extremos_meteorol%C3%B3gicos'

df_temperature: DataFrame = pd.read_html(WIKIPEDIA_COUNTRIES)[0]

display(df_temperature.head())
df_temperature.info()

,País/Região,Temperatura mais baixa,Local,Data,Temperatura mais alta,Local.1,Data.1
0,África do Sul,-20.1 ºC,"Buffelsfontein, perto de Molteno, Cabo Oriental",23 de agosto de 2013[1],50.0 ºC,"Dunbrody, Cabo Oriental",3 de novembro de 1918[2]
1,Argélia,-13.8 ºC,Mécheria,28 de janeiro de 2005[3],51.3 ºC,"El Bayadh, El Bayadh",2 de setembro de 1979[3]
2,Botswana,NaN,NaN,NaN,44.0 ºC,Maun,7 de janeiro de 2016[4]
3,Burquina Fasso,5 ºC,Markoye,janeiro de 1975[5],47.2 ºC,Dori,1984[5]
4,Chade,NaN,NaN,NaN,47.6 ºC,Faya-Largeau,22 de junho de 2010[6]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   País/Região             116 non-null    object
 1   Temperatura mais baixa  95 non-null     object
 2   Local                   94 non-null     object
 3   Data                    93 non-null     object
 4   Temperatura mais alta   110 non-null    object
 5   Local.1                 110 non-null    object
 6   Data.1                  109 non-null    object
dtypes: object(7)
memory usage: 6.5+ KB


In [6]:
df_temperature = df_temperature.dropna().reset_index(drop=True)

df_temperature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   País/Região             87 non-null     object
 1   Temperatura mais baixa  87 non-null     object
 2   Local                   87 non-null     object
 3   Data                    87 non-null     object
 4   Temperatura mais alta   87 non-null     object
 5   Local.1                 87 non-null     object
 6   Data.1                  87 non-null     object
dtypes: object(7)
memory usage: 4.9+ KB


In [7]:
COUNTRY_COLUMNS: dict[str, str] = {
    'País/Região': 'country',
    'Temperatura mais baixa': 'lowest_temp',
    'Temperatura mais alta': 'highest_temp'
}

df_ten_temperatures = df_temperature.sample(10)[COUNTRY_COLUMNS.keys()].rename(columns=COUNTRY_COLUMNS)

df_ten_temperatures.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 76 to 68
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country       10 non-null     object
 1   lowest_temp   10 non-null     object
 2   highest_temp  10 non-null     object
dtypes: object(3)
memory usage: 320.0+ bytes


In [8]:
def celsius_to_fahrenheit(celsius: float) -> float:
    return celsius * (9 / 5) + 32


def fahrenheit_to_celsius(fahrenheit: float) -> float:
    return fahrenheit - 32 * (5 / 9)

In [9]:
EXTRACT_TEMP: str = r'(-?\d{1,2}\.?\d?)'

df_ten_temperatures['highest_temp'] = df_ten_temperatures['highest_temp'].str.extract(EXTRACT_TEMP).astype(float)
df_ten_temperatures['lowest_temp'] = df_ten_temperatures['lowest_temp'].str.extract(EXTRACT_TEMP).astype(float)

df_ten_temperatures

,country,lowest_temp,highest_temp
76,Panamá,2.0,40.0
0,África do Sul,-20.1,50.0
26,Coreia do Sul,-32.8,40.0
22,Myanmar,-6.0,47.2
12,Hong Kong,-6.0,37.9
67,Suíça,-41.8,41.5
10,Camboja,10.0,42.6
18,Quirguistão,-53.6,44.0
4,Essuatíni,-6.7,47.7
68,Ucrânia,-41.9,42.0


In [10]:
df_ten_temperatures['lowest_temp_fahrenheit'] = df_ten_temperatures['lowest_temp'].apply(celsius_to_fahrenheit)
df_ten_temperatures['highest_temp_fahrenheit'] = df_ten_temperatures['highest_temp'].apply(celsius_to_fahrenheit)

df_ten_temperatures

,country,lowest_temp,highest_temp,lowest_temp_fahrenheit,highest_temp_fahrenheit
76,Panamá,2.0,40.0,35.60,104.00
0,África do Sul,-20.1,50.0,-4.18,122.00
26,Coreia do Sul,-32.8,40.0,-27.04,104.00
22,Myanmar,-6.0,47.2,21.20,116.96
12,Hong Kong,-6.0,37.9,21.20,100.22
67,Suíça,-41.8,41.5,-43.24,106.70
10,Camboja,10.0,42.6,50.00,108.68
18,Quirguistão,-53.6,44.0,-64.48,111.20
4,Essuatíni,-6.7,47.7,19.94,117.86
68,Ucrânia,-41.9,42.0,-43.42,107.60


### 1.2. Altura média por país

Tendo como dados de entrada a [altura média por país](https://pt.wikipedia.org/wiki/Estatura) crie um dicionário com 10 países, com chave como nome do país e valor como altura média, imprima em cada linha de um notebook o peso ideal:

$$
\text{Peso Ideal} = 72 \cdot \text{Altura} - 58
$$

por país, utilizando um dicionário e o operador de formatação [`f''`](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals)


In [14]:
WIKIPEDIA_AVG_HEIGHT: str = 'https://pt.wikipedia.org/wiki/Estatura'

df_heights: DataFrame = pd.read_html(WIKIPEDIA_AVG_HEIGHT)[1]

df_heights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   País                             84 non-null     object
 1   Altura média masculina           84 non-null     object
 2   Altura média feminina            74 non-null     object
 3   Grupo de idade toda a populaçao  84 non-null     object
 4   Método                           76 non-null     object
 5   Ano                              80 non-null     object
 6   Fonte                            84 non-null     object
dtypes: object(7)
memory usage: 4.7+ KB


In [65]:
HEIGHTS_COLUMNS: dict[str, str] = {
    'País': 'country',
    'Altura média masculina': 'avg_height_male',
    'Altura média feminina': 'avg_height_female'
}

df_ten_heights = df_heights.dropna().reset_index(drop=True)

df_ten_heights = df_ten_heights[HEIGHTS_COLUMNS.keys()].sample(10).rename(columns=HEIGHTS_COLUMNS)

df_ten_heights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 25 to 2
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            10 non-null     object
 1   avg_height_male    10 non-null     object
 2   avg_height_female  10 non-null     object
dtypes: object(3)
memory usage: 320.0+ bytes


In [66]:
HEIGHT_EXTRACT: str = r'(\d{2,3}[,\.]\d)'

df_ten_heights['avg_height_male'] = df_ten_heights['avg_height_male'].str.extract(HEIGHT_EXTRACT)[0].str.replace(',', '.').astype(float)
df_ten_heights['avg_height_female'] = df_ten_heights['avg_height_female'].str.extract(HEIGHT_EXTRACT)[0].str.replace(',', '.').astype(float)
df_ten_heights['avg_height'] = (df_ten_heights['avg_height_male'] + df_ten_heights['avg_height_female']) / 2

df_ten_heights

,country,avg_height_male,avg_height_female,avg_height
25,Cuba,168.0,155.0,161.50
0,Alemanha,179.0,166.0,172.50
11,Bolivia,166.6,155.4,161.00
56,Portugal,173.7,163.7,168.70
6,Austrália,177.8,163.8,170.80
47,Japão,171.4,158.2,164.80
30,Estados Unidos,178.0,164.5,171.25
57,Polônia,178.7,165.1,171.90
34,Estônia,181.2,168.2,174.70
2,Argélia,170.0,159.2,164.60


In [70]:
dict_heights: dict[str, float] = {row['country']: row['avg_height'] 
                                  for _idx, row in df_ten_heights.iterrows()}

dict_heights

{'Cuba': 161.5,
 'Alemanha': 172.5,
 'Bolivia': 161.0,
 'Portugal': 168.7,
 'Austrália': 170.8,
 'Japão': 164.8,
 'Estados Unidos': 171.25,
 'Polônia': 171.89999999999998,
 'Estônia': 174.7,
 'Argélia': 164.6}

In [62]:
def ideal_weight(height: float) -> float:
    return height * 72 - 58

In [77]:
for key, value in dict_heights.items():
    print(f'{key:>15} => Average height: {value:.2f} | Ideal weight: {ideal_weight(value) / 100:.2f}')

           Cuba => Average height: 161.50 | Ideal weight: 115.70
       Alemanha => Average height: 172.50 | Ideal weight: 123.62
        Bolivia => Average height: 161.00 | Ideal weight: 115.34
       Portugal => Average height: 168.70 | Ideal weight: 120.88
      Austrália => Average height: 170.80 | Ideal weight: 122.40
          Japão => Average height: 164.80 | Ideal weight: 118.08
 Estados Unidos => Average height: 171.25 | Ideal weight: 122.72
        Polônia => Average height: 171.90 | Ideal weight: 123.19
        Estônia => Average height: 174.70 | Ideal weight: 125.20
        Argélia => Average height: 164.60 | Ideal weight: 117.93


### 1.3. Conversão de valor de bolsa de estudos

Faça a conversão do valor de bolsas de pós-graduação de [três países](https://exame.abril.com.br/carreira/confira-63-bolsas-para-estudar-em-paises-como-canada-eua-e-alemanha/) para Reais, imprimindo o salário de uma jornada de 40h semanais:
* Por hora trabalhada;
* Por mês;
* Por ano.


In [81]:
CURRENCY_API_URL: str = 'https://economia.awesomeapi.com.br/json/last/'

CURRENCIES: dict[str, str] = {
    'USD': 'USD-BRL',
    'EUR': 'EUR-BRL'
}

def get_currency_for(currency: str) -> float:
    try:
        response: Response = req_get(f'{CURRENCY_API_URL}/{currency}')
        
        if response.status_code != 200:
            raise Exception('Response was not successfully completed')
        
        return float(response.json()[currency.replace('-', '')]['bid'])
        
    except Exception:
        return 0


> [...]
> 
> **CAPES e DAAD oferecem bolsas para doutorado na Alemanha**
> 
> A CAPES e o DAAD (serviço alemão de intercâmbio acadêmico) estão recebendo inscrições para bolsas de doutorado na Alemanha. Estão disponíveis bolsas tanto para doutorado pleno quanto para estadias mais curtas, como doutorado sanduíche ou doutorado com cotutela. As inscrições vão até 20 de dezembro.
> 
> São até 20 bolsas para doutorado pleno (com duração de até quatro anos), 22 bolsas para doutorado sanduíche curto (entre quatro e 12 meses), 15 bolsas para doutorado sanduíche longo (de 12 a 24 meses) e oito bolsas para o doutorado com cotutela (entre quatro e 18 meses). Os bolsistas receberão um valor mensal de 1.200 euros para arcar com os custos de vida na Alemanha. A bolsa também cobre o seguro saúde para os estudantes e oferece um auxílio de 1.575 euros para despesas com passagem aérea. Esse último valor será depositado na Alemanha junto com a primeira mensalidade da bolsa. O edital também cita um auxílio-instalação e um auxílio-dependente (este último apenas para doutorado pleno), cujos valores não são especificados.
> 
> [...]

In [85]:
MONTHLY: float = 1200. # 1200 por mês

EURO_RATE: float = get_currency_for(CURRENCIES['EUR'])

print('Hour (40h week): R$ {:,.2f}'.format((MONTHLY / (24 * 30)) * EURO_RATE))
print('Month: R$ {:,.2f}'.format(MONTHLY * EURO_RATE))
print('Year: R$ {:,.2f}'.format((MONTHLY * EURO_RATE) * 12))

Hour (40h week): R$ 9.85
Month: R$ 7,091.40
Year: R$ 85,096.80


> [...]
> 
> **Bolsas de estudo para mestrado em saúde pública**
> 
> Estão abertas as candidaturas para o programa “EUROPUBHEALTH” de mestrado em saúde pública na Europa, promovido pela Comissão Europeia. As inscrições para as bolsas de excelência disponíveis (cuja quantidade ainda não foi informada) podem ser realizadas até 15 de janeiro, pelo site.
> 
> Para o ano letivo 2020-2022, serão distribuídas bolsas de excelência no valor de, no máximo, 46 mil euros pelos dois anos, além de bolsas adicionais especialmente destinadas aos melhores alunos de algumas regiões prioritárias – incluindo América Latina. Esse valor cobre os custos de participação no programa,até três mil euros em passagens aéreas por ano, um auxílio-instalação de mil euros, e uma mesada de mil euros para ajudar com o custo de vida.
> 
> [...]

In [86]:
MONTHLY: float = 1000. # 1200 por mês

EURO_RATE: float = get_currency_for(CURRENCIES['EUR'])

print('Hour (40h week): R$ {:,.2f}'.format((MONTHLY / (24 * 30)) * EURO_RATE))
print('Month: R$ {:,.2f}'.format(MONTHLY * EURO_RATE))
print('Year: R$ {:,.2f}'.format((MONTHLY * EURO_RATE) * 12))

Hour (40h week): R$ 8.21
Month: R$ 5,909.50
Year: R$ 70,914.00


> [...]
> 
> **Bolsas de estudo integrais de pós em Abu Dhabi**
> 
> A Khalifa University está recebendo inscrições para bolsas integrais de pós-graduação em Abu Dhabi, onde está sediada. As bolsas contemplam tanto os programas 14 de mestrado > quanto os 12 de doutorado oferecidos pela universidade. As inscrições vão até 3 de novembro para ingresso em janeiro de 2020, ou até fevereiro de 2020 para ingresso em agosto.
> 
> A lista completa dos programas oferecidos pela instituição pode ser vista neste link, sob a aba “Academic”. As bolsas integrais para pós-graduação em Abu Dhabi cobrem totalmente os custos de tuition do programa e incluem passagens aéreas anuais. Também cobrem seguro de saúde e um valor de US$ 2.200 a US$ 2.700 por mês para ajuda com os custos de vida no país.
> 
> [...]

In [88]:
MONTHLY: float = ((2200 + 2700) / 2) # Média de 2200 e 2700 por mês

EURO_RATE: float = get_currency_for(CURRENCIES['USD'])

print('Hour (40h week): R$ {:,.2f}'.format((MONTHLY / (24 * 30)) * EURO_RATE))
print('Month: R$ {:,.2f}'.format(MONTHLY * EURO_RATE))
print('Year: R$ {:,.2f}'.format((MONTHLY * EURO_RATE) * 12))

Hour (40h week): R$ 18.68
Month: R$ 13,450.75
Year: R$ 161,408.94


## 2. Estruturas de Decisão

### 2.1. Está frio?

Escreva código em uma célula, para, dado o elemento de uma lista de temperaturas acima, imprimir "`está frio`" se a temperatura for menor que um valor de uma variável `t` definida por você ou "`está calor`" caso contrário.

### 2.2. Combustíveis

Um posto está vendendo combustíveis com a seguinte tabela de descontos:

|  Volume  |         Álcool         |        Gasolina        |
|---------:|:----------------------:|:-----------------------|
|$\lt{20l}$|desconto de $3%$ por $l$|desconto de $4%$ por $l$|
|$\ge{20l}$|desconto de $5%$ por $l$|desconto de $6%$ por $l$|

Escreva um código que leia o número de litros vendidos, o tipo de combustível (codificado da seguinte forma: A-álcool, G-gasolina), calcule e imprima o valor a ser pago pelo cliente sabendo-se que o preço do litro da gasolina é 2,50 reais, e o preço do litro do álcool é de 1,90 reais.


### 2.3. Frutaria

Uma fruteira está vendendo frutas com a seguinte tabela de preços:

|Fruta|$\lt{5}\text{Kg}$|$\ge{5}\text{Kg}$|
|-:|:-:|:-|
|Morango|2,50 reais por kilo|2,20 reais por kilo|
|Maçã|1,80 reais por kilo|1,50 reais por kilo|

Se o cliente comprar mais de 8 Kg em frutas ou o valor total da compra ultrapassar R$ 25,00, receberá ainda um desconto de 10% sobre este total. Escreva código para ler a quantidade (em Kg) de morangos e a quantidade (em Kg) de maças adquiridas e escreva o valor a ser pago pelo cliente.

## 3. Estruturas de Repetição

### 3.1. Tabuada

Desenvolva um gerador de tabuada, capaz de gerar a tabuada de qualquer número inteiro entre 1 a 10. Dada uma variável k em uma célula calcular a tabuada para o número. A saída deve ser conforme o exemplo abaixo:

```text
Tabuado de 5:
5 X 1 = 5
5 X 2 = 10
...
5 X 10 = 50
```

### 3.2. Repetição com temperaturas

Utilize uma estrutura de repetição para imprimir todos os elementos das listas de temperatura e altura criadas acima.

### 3.3. Repetindo e formatando

Acesse chaves e valores do dicionário criado acima, e imprima cada item utilizando o operador de formatação.

## 4. Funções

### 4.1. Invertendo

Faça uma função que retorne o reverso de um número inteiro informado.
> Por exemplo: `127 -> 721`

### 4.2. Convertendo uma temperatura

Faça uma função para converter, que dado um valor numérico de temperatura e um valor de caractere para a escala (Celsius ou Farenheit) converta o número para a escala desejada e retorna o valor.

### 4.3. 24 para 12 horas com AM/PM

Faça uma programa que converta da notação de 24 horas para a notação de 12 horas. Por exemplo, o programa deve converter 14:25 em 2:25 P.M. A entrada é dada em dois inteiros.

## 5. NumPy

### 5.1. Primeira matriz NumPy

Escreva código para criar uma matriz 4 x 10, com números simulados de uma distribuição normal com média 10 e variância 5, e substitua todos os elementos da primeira coluna maiores que 12 por 0.

### 5.2. Alturas, Pesos e Matrizes

Crie uma matriz 3x3 com 3 alturas de 3 países e calcule o peso ideal (fórmula acima) para multiplicar e somar aos elementos da matriz.

### 5.3. NumPy the Hardway: No broadcasting 💪

Faça uma função que recebe dois parâmetros, uma matriz numpy e uma constante, e utiliza estruturas de repetição para multiplicar cada elemento da matriz e retorna a matriz multiplicada.

## 6. Pandas

> Para os exercícios, utilize [este conjunto de dados](https://drive.google.com/open?id=1u1SEDTja9QC7MAj_VlQcxNtAhQfs-5YR)

### 6.1. Explorando o [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

Faça uma função, que dado um dataframe com os dados acima. imprime a média e o desvio padrão das variáveis 'season' e 'holiday'.

### 6.2. Novas colunas, novos dados

Crie uma nova coluna no `dataframe`, com o nome `'month'` com o mês correspondente de cada linha da coluna `'datetime'` e escreva uma função que dado um `dataframe`, e um valor numérico entre 1 e 12, retorna um sub-`dataframe` para o mês requerido.

### 6.3. _Histogramando_ um pouco

Faça um histograma da variável `'temp'` e um boxplot único com a variável `'temp'` em cada `'month'` (no eixo x).


## 7. Regressão

> **Incluir o conjunto de dados na entrega do notebook.**
>
> **Os conjuntos de dados devem ser diferentes entre os alunos**

**Dataset escolhido:** [Apartment for Rent Classified](https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified)

Em uma célula do mesmo notebook crie um texto com markdown, incluindo formatação com títulos, **negrito**, _itálico_ e inclusão de figuras para explicar:


* Explique simplificadamente o que é o modelo de regressão e quando estes modelos podem ser utilizados;

* Apresente um pequeno exemplo numérico utilizando o que foi aprendido no curso, utilizando um conjunto de dados que não foi visto no curso (pode ser de qualquer lugar na internet) e imprimindo os seguintes resultados:
  * [R2](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html)
  * [Coeficientes da regressão](https://www.geeksforgeeks.org/regression-coefficients/)

* E fazendo um gráfico com a variável resposta (eixo `y`) uma das variáveis preditoras (eixo `x`) e a reta da regressão;

## 8. Classificação

> **Incluir o conjunto de dados na entrega do notebook.**
>
> **Os conjuntos de dados devem ser diferentes entre os alunos**

**Dataset escolhido**: [Heart Failure Clinical Records](https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records)

Em uma célula do mesmo notebook crie um texto com markdown, incluindo formatação com títulos, **negrito**, _itálico_ e inclusão de figuras para explicar:


* Explique simplificadamente o que é o modelo de classificação e quando estes modelos podem ser utilizados;

* Apresente um pequeno exemplo numérico utilizando o que foi aprendido no curso, utilizando um conjunto de dados que não foi visto no curso (pode ser de qualquer lugar na internet) e imprimindo, utilizando [validação cruzada](https://scikit-learn.org/stable/modules/cross_validation.html), os seguintes resultados:
  * [Acurácia](https://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score)
  * [Matriz de confusão](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)

## 9. Agrupamento

> **Incluir o conjunto de dados na entrega do notebook.**
>
> **Os conjuntos de dados devem ser diferentes entre os alunos**

**Dataset escolhido:** [Turkish Music Emotion](https://archive.ics.uci.edu/dataset/862/turkish+music+emotion)

Em uma célula do mesmo notebook crie um texto com markdown, incluindo formatação com títulos, **negrito**, _itálico_ e inclusão de figuras para explicar:


* Explique simplificadamente o que é o modelo de [Análise de Componentes Principais (_PCA_)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#pca) e quando estes modelos podem ser utilizados;

* Apresente um pequeno exemplo numérico utilizando o que foi aprendido no curso, utilizando um conjunto de dados que não foi visto no curso (pode ser de qualquer lugar na internet) e imprimindo, os seguintes resultados:
  * Variância explicada por cada componente

* Faça um gráfico com os _scores_ do _PCA_ colorindo pelos grupos do seu conjunto de dados.

## 10. Referências

* [Python Brasil - Lista de Exercícios](https://wiki.python.org.br/ListaDeExercicios)
* [numpy](https://numpy.org/doc/stable/reference/index.html)
* [matplotlib](https://matplotlib.org/stable/api/index)
* [seaborn](https://seaborn.pydata.org/api.html)
* [pandas](https://pandas.pydata.org/docs/reference/index.html#api)
* [scikit-learn](https://scikit-learn.org/stable/api/index.html)
* Datasets:
  * Regressão:
    * Apartment for Rent Classified. (2019). UCI Machine Learning Repository. https://doi.org/10.24432/C5X623.
  * Classificação:
    * Heart Failure Clinical Records. (2020). UCI Machine Learning Repository. https://doi.org/10.24432/C5Z89R.
  * Agrupamento:
    * Er,Mehmet Bilal. (2023). Turkish Music Emotion. UCI Machine Learning Repository. https://doi.org/10.24432/C5JG93.